In [1]:
using LinearAlgebra

## First bad idea for eigenvalues: roots of the characteristic polynomial

Even if the eigevalues of $A$ are not sensitive to perturbations of $A$, the roots of $p(A)$ may be very sensitive to perturbtations of its coefficients.

In [2]:
A = [ 1 0; 0 nextfloat(1.)]

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

In [3]:
using Polynomials
p = Poly([1*nextfloat(1.),-1-nextfloat(1.),1])

Poly(1.0000000000000002 - 2.0*x + 1.0*x^2)

In [4]:
roots(p)

2-element Array{Complex{Float64},1}:
 1.0 + 1.490116119384766e-8im
 1.0 - 1.490116119384766e-8im

## Second bad idea for eigenvalues

Let's try our usual tricks to introduce zeros and make $A$ diagonal. We start with a matrix that has very recognizable eigenvalues.

In [5]:
m=5;  V = randn(m,m)
A = V\(Diagonal(1:m)*V)
λ = sort( eigvals(A) )

5-element Array{Float64,1}:
 0.9999999999999968
 2.0000000000000013
 3.0000000000000044
 4.0               
 5.000000000000006 

In [6]:
function hh(x)
    v = copy(x)
    v[1] += sign(x[1]+1e-300)*norm(x)
    return v/norm(v)
end

hh (generic function with 1 method)

In [7]:
#V = zeros(m,m)
v = fill([],m-1)
for j = 1:m-1
    v[j] = hh(A[j:m,j])
    A[j:m,:] -= 2*v[j]*(v[j]'*A[j:m,:])
#    V[j:m,j] = hh(A[j:m,j])
#    A[j:m,:] -= 2*V[j:m,j]*(V[j:m,j]'*A[j:m,:])
end

So far, so good. We have introduced a lot of zeros.

In [8]:
A

5×5 Array{Float64,2}:
 -4.38113  -4.03559      -1.27734       5.20735      -1.38448 
  0.0      -9.55664      -1.31968       6.37867      -3.03821 
  0.0       2.2222e-15   -1.40568       6.2527        0.382261
  0.0      -2.89275e-16  -1.51029e-16  -9.71958       0.168739
  0.0      -1.13536e-15   4.29204e-17  -3.55271e-15   0.209776

However, we changed the eigenvalues.

In [9]:
eigvals(A)

5-element Array{Float64,1}:
 -4.381127369816106  
 -9.556644434058494  
 -9.719581117477365  
  0.20977569168615548
 -1.405682468616343  

To preserve eigenvalues, we have to operate with similarity transformations. This means some right-multiplications.

In [10]:
for j = 1:m-1
    A[:,j:m] -= 2*(A[:,j:m]*v[j])*v[j]'
end

In [11]:
sort( eigvals(A) )

5-element Array{Float64,1}:
 1.0000000000000004
 2.000000000000003 
 2.999999999999997 
 3.9999999999999973
 5.000000000000009 

But we wrecked the zero structure!

In [12]:
A

5×5 Array{Float64,2}:
  1.18496     1.93905   -1.66188     -1.48145    7.48496  
 -3.45025     6.85262   -4.35824     -2.39647    7.70644  
 -1.76395     1.04753    3.24001     -1.15029    5.01883  
  3.06886    -0.780739  -4.22161      3.6855    -7.28466  
 -0.0874571  -0.123343   0.00965244   0.140314   0.0369114

## On the verge of a great idea

On a whim, let's do the previous bad idea over and over. At least it won't change the eigenvalues, right? 

In [13]:
m=7;  V = randn(m,m)
A = V\(Diagonal(1:m)*V)

7×7 Array{Float64,2}:
  3.0808     -0.0374204  -0.116901   -1.19697  -3.58626    1.16771    1.32354
 -0.952021    3.67571    -0.0309002  -2.67003  -4.54425    2.03159   -0.1506 
  3.55869     9.83639     9.02204     8.04683  21.2862   -28.0422   -18.0548 
  2.00419     2.10444     1.51449     7.45717   8.10497  -10.146     -6.97405
 -0.129475   -1.72879    -0.43679    -1.0845    2.81943    2.95127    1.24215
  0.583484    1.32691     1.12517     1.66535   4.95738   -2.65248   -3.47803
 -0.0254614  -2.00782    -0.75393    -1.50946  -3.42843    3.30357    4.59732

In [14]:
function dumb(AA)
    A = copy(AA)
    v = fill([],m-1)
    # put in zeros
    for j = 1:m-1
        v[j] = hh(A[j:m,j])
        A[j:m,:] -= 2*v[j]*(v[j]'*A[j:m,:])  
    end
    # restore eigenvalues
    for j = 1:m-1
        A[:,j:m] -= 2*(A[:,j:m]*v[j])*v[j]'
    end

    return A
end

dumb (generic function with 1 method)

In [15]:
sort(eigvals(dumb(A)))

7-element Array{Float64,1}:
 0.9999999999999996
 1.9999999999999987
 3.0000000000000036
 4.000000000000002 
 4.999999999999995 
 6.000000000000004 
 6.9999999999999964

In [16]:
B = copy(A)
for i = 1:10
    global B=dumb(B)
end
sort(eigvals(B))

7-element Array{Float64,1}:
 0.999999999999999 
 1.9999999999999976
 2.9999999999999956
 3.9999999999999942
 4.999999999999996 
 5.999999999999951 
 7.000000000000028 

In [17]:
B

7×7 Array{Float64,2}:
  8.36659       3.51908      -4.31107     …   35.8971       1.84398  
 -0.917802      4.64396       1.98026        -18.756        0.822301 
  0.0295291     0.0867055     5.04774         -0.307781     0.360667 
  0.01275       0.0347208     0.0558727        0.378341    -0.137316 
  5.02653e-5   -0.000126907   0.0213101        0.144314    -0.459524 
  2.47818e-6    8.64731e-6    2.98012e-5  …    1.99904     -0.0774645
 -3.87536e-10  -8.77685e-9    5.20117e-8      -0.00125426   1.0001   

Remember, we're putting zeros in the lower triangle and then filling them back in. But some of those entries are staying pretty small.

In [18]:
@. round(Int,log(10,abs(B)))

7×7 Array{Int64,2}:
  1   1   1   0   1   2   0
  0   1   0   0   1   1   0
 -2  -1   1   0   0  -1   0
 -2  -1  -1   1  -1   0  -1
 -4  -4  -2  -1   0  -1   0
 -6  -5  -5  -3  -2   0  -1
 -9  -8  -7  -6  -5  -3   0

In [19]:
B = copy(A)
for i = 1:400
    global B=dumb(B)
end
@. round(Int,log(10,abs(B)))

7×7 Array{Int64,2}:
    1     1     1     0     1     2   0
  -27     1    -1     0     1     1   0
  -59   -32     1     0    -1    -1  -1
  -97   -70   -39     1    -2     0  -1
 -148  -121   -88   -50     0    -1   0
 -218  -191  -160  -121   -71     0  -1
 -322  -311  -280  -240  -191  -120   0

This matrix is virtually triangular! So the eigenvalues are just sitting there on the diagonal.

In [20]:
diag(B)

7-element Array{Float64,1}:
 7.000000000000064 
 5.9999999999999485
 4.999999999999985 
 3.999999999999997 
 2.9999999999999987
 1.9999999999999956
 0.9999999999999992

They even came out sorted!?!